In [1]:
import pandas as pd

In [ ]:
from google.colab import drive  
drive.mount('/content/drive')

In [3]:
dir = "/content/drive/MyDrive/Magang M-Knows/sentiment_analysis/"

In [4]:
df_clean = pd.read_csv(dir+'data/df_clean.csv', index_col=0)
df_clean.head()

,comment,clean
0,Izin bertnya apakah studi idenpenden mitra pro...,izin bertnya studi idenpenden mitra programing...
1,Dapat uang saku gaksih,uang saku gaksih
2,@rwrt1.0 kelas full english kak batch ini,kelas full english batch
3,"Saya, saya bang ga lolos msib",lolos msib
4,🔥🔥,NaN


In [5]:
# remove rows with missing values
df_clean = df_clean.dropna()

## Labeling

### TextBlob

In [6]:
from textblob import TextBlob

In [7]:
# Menggunakan TextBlob untuk melakukan sentiment analysis
def get_polarity(text):
    blob = TextBlob(text)

    # Translate kalimat ke bahasa Inggris
    try:
        an = blob.translate(from_lang='id', to='en')
        sentimen = an.sentiment.polarity
    except Exception as e:
        sentimen = blob.sentiment.polarity
    
    # Memberikan label sentimen berdasarkan nilai polaritas
    if sentimen > 0:
        label_sentimen = "Positif"
    elif sentimen == 0:
        label_sentimen = "Netral"
    else:
        label_sentimen = "Negatif"
        
    return label_sentimen

In [8]:
df_clean['sentimen'] = df_clean['clean'].apply(get_polarity)

In [9]:
# count the number of each sentiment
print(df_clean['sentimen'].value_counts())

Netral     285
Positif    122
Negatif    111
Name: sentimen, dtype: int64


## IndoBERT

In [17]:
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

In [19]:
pretrained= "mdhugol/indonesia-bert-sentiment-classification"
model = AutoModelForSequenceClassification.from_pretrained(pretrained)
tokenizer = AutoTokenizer.from_pretrained(pretrained)

In [20]:
sentiment_analysis = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

In [30]:
label_index = {'LABEL_0': 'Positif', 'LABEL_1': 'Netral', 'LABEL_2': 'Negatif'}

In [31]:
# test sentiment
pos_text = "aku ganteng"

result = sentiment_analysis(pos_text)
status = label_index[result[0]['label']]
score = result[0]['score']
print(f'Text: {pos_text} | Label : {status} ({score * 100:.3f}%)')

Text: aku ganteng | Label : Positif (99.529%)


In [32]:
def get_sentiment_indobert(text):
    result = sentiment_analysis(text)
    status = label_index[result[0]['label']]
    return status

In [41]:
df_clean['sentiment_indobert'] = df_clean['clean'].apply(get_sentiment_indobert) 

In [36]:
df_clean['sentiment_indobert_kotor'] = df_clean['comment'].apply(get_sentiment_indobert)

In [53]:
df_clean.sample(10)

,comment,clean,sentimen,sentiment_indobert,sentiment_indobert_kotor
46,Perpanjang lgi dong minnnn,panjang minnnn,Negatif,Negatif,Negatif
80,@mas.paijho benar bung semangat,semangat,Netral,Positif,Positif
154,@dianicoputra aminnn,amin,Netral,Positif,Positif
135,"@theorifai Loh iya memang😂, tapi buktinya bany...",loh bukti sungguh gagal gagal berkas gara sist...,Negatif,Negatif,Negatif
489,@bochiillll @cicumgarcum kut yo,kut,Netral,Netral,Netral
541,"@ema.hakimah kalo hanya simulasi, loginnya har...",simulasi loginnya registrasi survei pakai user...,Netral,Netral,Netral
384,tolong untuk kedepan bagi yg sudah pernah meng...,tolong depan ikut magang kampus merdeka tement...,Netral,Netral,Negatif
319,@kriscokk magang msib biasanya brp lama bang?,magang msib,Netral,Positif,Netral
438,"@yunpnzr ya mas, mitra ama PIC kampus aja gk b...",mitra pic kampus bantu ngecekin tuntut transpa...,Netral,Negatif,Negatif
207,@theorifai Kenapa ga mengubah mainset untuk me...,ubah mainset sudut pandang dokumen salah bukti...,Negatif,Netral,Negatif


In [54]:
# save the data
df_clean.to_csv(dir+'data/df_sentiment.csv')

### Indonesia RoBERTa

In [56]:
from transformers import pipeline

pretrained_name = "w11wo/indonesian-roberta-base-sentiment-classifier"

roberta = pipeline(
    "sentiment-analysis",
    model=pretrained_name,
    tokenizer=pretrained_name
)

In [106]:
def get_sentiment_roberta(text):
    result = roberta(text)
    status = result[0]['label']
    return status

In [108]:
df_clean['sentiment_roberta'] = df_clean['clean'].apply(get_sentiment_roberta) 

In [109]:
df_clean['sentiment_roberta_kotor'] = df_clean['comment'].apply(get_sentiment_roberta) 

In [110]:
cols = df_clean.columns
cols = cols[2:]
cols

Index(['sentimen', 'sentiment_indobert', 'sentiment_indobert_kotor',
       'sentiment_roberta', 'sentiment_roberta_kotor'],
      dtype='object')

In [111]:
for col in cols:
  print(col)
  print(df_clean[col].value_counts())
  print()

sentimen
Netral     285
Positif    122
Negatif    111
Name: sentimen, dtype: int64

sentiment_indobert
Negatif    226
Netral     193
Positif     99
Name: sentiment_indobert, dtype: int64

sentiment_indobert_kotor
Netral     255
Negatif    219
Positif     44
Name: sentiment_indobert_kotor, dtype: int64

sentiment_roberta
neutral     249
negative    169
positive    100
Name: sentiment_roberta, dtype: int64

sentiment_roberta_kotor
negative    303
neutral     165
positive     50
Name: sentiment_roberta_kotor, dtype: int64



In [112]:
df_clean.sample(10)

,comment,clean,sentimen,sentiment_indobert,sentiment_indobert_kotor,sentiment_roberta,sentiment_roberta_kotor
336,@ooo_adhe ada aku ketrima bg,terima,Netral,Positif,Netral,positive,positive
286,@theresiaaclaudiaa hah jadinya maksudnya bisa ...,hah maksud tolak accord web masuk grup plis bi...,Netral,Negatif,Negatif,negative,negative
219,@theorifai iyhh bg abang suratnya di acc soaln...,iyhh abang surat accord enggak tau bikin surat...,Netral,Negatif,Negatif,negative,negative
302,Sedih sebenarnya karena yang diikutkan hanya p...,sedih ikut guru bawah kemenristekdikti milik a...,Negatif,Negatif,Negatif,neutral,negative
199,@theorifai mereka beruntung sih gak kena UU pe...,untung kena uu palsu dokumen,Negatif,Negatif,Positif,negative,positive
14,Heh nak bangkit @koo.chiko,heh nak bangkit,Netral,Negatif,Negatif,negative,negative
87,@bro.swryz ada plus minus kok dgn sistem ini. ...,plus minus sistem daftar mendadak2 submitnya p...,Negatif,Negatif,Netral,negative,negative
329,"Terpaksa mengambil Studi Independen, hanya kar...",paksa ambil studi independen kejar deadline ke...,Negatif,Negatif,Negatif,negative,negative
570,Ini untuk dosen PT mahasiswa yang ikut msib ba...,dosen pt mahasiswa msib batch,Netral,Netral,Netral,neutral,neutral
135,"@theorifai Loh iya memang😂, tapi buktinya bany...",loh bukti sungguh gagal gagal berkas gara sist...,Negatif,Negatif,Negatif,negative,negative


In [113]:
df_clean.to_csv(dir+"data/df_clean.csv")